In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab4.ipynb")

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
%matplotlib inline

## Objective

In this lab you will get some practice plotting, applying data transformations, and working with kernel density estimators.  We will be working with data from the World Bank containing various statistics for countries and territories around the world.  

## Loading Data

In [2]:
wb = pd.read_csv("data/world_bank_misc.csv", index_col=0)
wb.head()

This table contains some interesting columns.  Take a look:

In [3]:
list(wb.columns)

# Part 1: Scaling

In the first part of this assignment we will look at the distribution of values for female adult literacy rate as well as the gross national income per capita. The code below creates a copy of the dataframe that contains only the two Series we want, and then drops all rows that contain null values in either column. 

In [4]:
#creates a dataframe with the appropriate index
df = pd.DataFrame(index=wb.index)

#copies the Series we want
df['lit'] = wb['Adult literacy rate: Female: % ages 15 and older: 2005-14']
df['inc'] = wb['Gross national income per capita, Atlas method: $: 2016']

#the line below drops all records that have a NaN value in either column
df.dropna(inplace=True)
print("Original records:", len(wb))
print("Final records:", len(df))

In [5]:
df.head(5)

## Question 1a

Suppose we wanted to build a histogram of our data to understand the distribution of literacy rates and income per capita individually. Last week, we saw that `countplot` creates histograms from categorical data. 

In [6]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.countplot(data=df, x='lit')
plt.xlabel("Adult literacy rate: Female: % ages 15 and older: 2005-14")
plt.title('World Bank Female Adult Literacy Rate')

plt.subplot(1,2,2)
sns.countplot(data=df, x='inc')
plt.xlabel('Gross national income per capita, Atlas method: $: 2016')
plt.title('World Bank Gross National Income Per Capita')
plt.show()

<!-- BEGIN QUESTION -->

In the cell below, concisely explain why `countplot` is NOT the right tool for the job.

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 3
-->

*Your explanation here...*

An alternate type of plot is the `barplot`, which as you'll see below, provides some vague idea of the distribution, but this is also not what we want.

In [7]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.barplot(data=df, x='lit')
plt.xlabel("Adult literacy rate: Female: % ages 15 and older: 2005-14")
plt.title('World Bank Female Adult Literacy Rate')

plt.subplot(1,2,2)
sns.barplot(data=df, x='inc')
plt.xlabel('Gross national income per capita, Atlas method: $: 2016')
plt.title('World Bank Gross National Income Per Capita')
plt.show()

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 1b

In the cell below, create a plot of literacy rate and income per capita using the `histplot` function. As above, you should have two subplots, where the left subplot is literacy, and the right subplot is income. When you call `histplot`, set the `kde` parameter to false, e.g. `histplot(s, kde=False)`.

Don't forget to title the plot and label axes!

**Hint:** *Copy and paste from above to start.*

<!--
BEGIN QUESTION
name: q1b1
manual: true
points: 1
-->

In [8]:
...

<!-- END QUESTION -->



You should see histograms that show the counts of how many data points appear in each bin. `histplot` uses a heuristic called the Freedman-Diaconis rule to automatically identify the best bin sizes, though it is possible to set the bins yourself (we won't).

Binning the data loses some information about the individual data points. A *rugplot* is a way to add this data back into the plot. Notice below how we add a rugplot to the histogram. Feel free to experiment with the settings to make a nicer plot.

In [9]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.histplot(data=df, x='lit', kde=False)
sns.rugplot(data=df, x='lit', height=-0.02, clip_on=False)
plt.xlabel("Adult literacy rate: Female: % ages 15 and older: 2005-14")
plt.title('World Bank Female Adult Literacy Rate')
plt.ylabel('Count')

plt.subplot(1,2,2)
sns.histplot(data=df, x='inc', kde=False)
sns.rugplot(data=df, x='inc', height=-0.02, clip_on=False)
plt.xlabel('Gross national income per capita, Atlas method: $: 2016')
plt.title('World Bank Gross National Income Per Capita')
plt.ylabel('Count')
plt.show()

<!-- BEGIN QUESTION -->

In the cell below, let's do one last tweak and plot with the `kde` parameter set to True. Further, add the `stat='density'` argument to the `histplot`s.

<!--
BEGIN QUESTION
name: q1b2
manual: true
points: 1
-->

In [10]:
...

<!-- END QUESTION -->



You should see roughly the same histogram as before. However, now you should see an overlaid smooth line. This is the kernel density estimate discussed in class. 

Observations:
* You'll also see that the y-axis value is no longer the count. Instead it is a value such tha the total area under the KDE curve is 1. The KDE estimates the underlying probability density function of the given variable. 
* The KDE is just an estimate, and we can see that it makes some silly decisions, e.g. assigning a non-zero probability of a greater than 100% literacy rate. 

We'll talk more about KDEs later in this lab.

<!-- BEGIN QUESTION -->

## Question 1c

Looking at the income data, it is difficult to see the distribution among high income (> $30000) countries, and the KDE also has a funny artifact where the probability density function has little bumps just above zero that correspond to the wealthy outlying countries.

We can logarithmically scale the x-axis to give us a visual representation that makes it easier to see patterns and also give a more reasonable KDE.

In the cell below, make a distribution plot with the data transformed using `log_scale=True` with `kde=True` and `stat='density'`. Be sure to correct the axis label using `plt.xlabel`. If you want to see the exact counts, just change the `stat` argument.
<!--
BEGIN QUESTION
name: q1c
manual: true
points: 3
-->

In [11]:
plt.figure()
...

...

<!-- END QUESTION -->



# Part 2: Kernel Density Estimation

In this part of the lab you will develop a deeper understanding of how kernel destiny estimation works.

Suppose we have 3 data points with values 2, 4, and 9. We can compute the (useless) histogram as shown below.

In [12]:
data3pts = np.array([2, 4, 9])
sns.histplot(data3pts, kde=False);

By setting `kde=True`, we can see a kernel density estimate of the data.

In [13]:
sns.histplot(data3pts, kde = True);

One question you might be wondering is how the kernel density estimator decides how "wide" each point should be. It turns out this is a parameter you can adjust, called *bandwith*. For example, the code below adjusts the bandwith value by 0.15 at each data point. You'll see the resulting kde is quite different. Try experimenting with different values of bandwidth and see what happens. (*Note: The `cut` argument allows the curves to extend past the data*).

In [14]:
sns.histplot(data3pts, kde = True, kde_kws = {"bw_adjust": 0.15, "cut": 5});

## Question 2a

As mentioned in class, the kernel density estimate is just the sum of a bunch of copies of the kernel, each centered on our data points. For those of you familiar with the idea of "convolution", the KDE is simply the convolution of the kernel with the data. The default kernel used by the `histplot` function is the Guassian kernel, given by:

$$\Large
K_\alpha(x, z) = \frac{1}{\sqrt{2 \pi \alpha^2}} \exp\left(-\frac{(x - z)^2}{2  \alpha ^2} \right)
$$

In Python code, this function is given as below, where `alpha` is the parameter $\alpha$, `z` is the x coordinate of the center of the Gaussian (i.e. a data point), and `x` is the independent variable.

In [15]:
def gaussian_kernel(alpha, x, z):
    return 1.0/np.sqrt(2. * np.pi * alpha**2) * np.exp(-(x - z) ** 2 / (2.0 * alpha**2))

For example, we can plot the gaussian kernel centered on $x$ coordinate 9 with $\alpha$ = 0.5 as below: 

In [16]:
xs = np.linspace(-2, 12, 200)
alpha=0.5
kde_curve = [gaussian_kernel(alpha, x, 9) for x in xs]
plt.plot(xs, kde_curve);

<!-- BEGIN QUESTION -->

In the cell below, plot the 3 kernel density functions corresponding to our 3 data points on the same axis. Use an `alpha` value of 0.5.

**Hint:** *The `gaussian_kernel` function can take a numpy array as an argument for `z`*.

<!--
BEGIN QUESTION
name: q2a1
manual: true
points: 1
-->

In [17]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

In the cell below, create a plot showing the sum of all three of the kernels above. Your plot should closely resemble the kde shown when you called `histplot` function with bandwidth 0.5 earlier.

**Hint: ** *Consider using np.sum with the argument `axis = 1`.*

**Hint: ** *Make sure to normalize your kernels!*

<!--
BEGIN QUESTION
name: q2a2
manual: true
points: 2
-->

In [18]:
...

<!-- END QUESTION -->



## Question 2b

Recall that earlier we plotted the kernel density estimation for the logarithm of the income data, as shown again below.

In [19]:
ax = sns.histplot(data=df, x='inc', kde=True, stat='density', log_scale=True)
plt.title('World Bank Gross National Income Per Capita')
plt.xlabel('Log Gross national income per capita, Atlas method: $: 2016');

<!-- BEGIN QUESTION -->

In the cell below, make a similar plot using your technique from question 2a. Give an estimate of the $\alpha$ value chosen by the `sns.histplot` function by tweaking your `alpha` value until your plot looks almost the same.

<!--
BEGIN QUESTION
name: q2b
manual: true
points: 3
-->

In [20]:
...

<!-- END QUESTION -->



## Question 2c

In your answers above, you hard coded a lot of your work. In this problem, you'll build a more general kernel density estimator function.

Implement the KDE function which computes:

$$\Large
f_\alpha(x) = \frac{1}{n} \sum_{i=1}^n K_\alpha(x, z_i)
$$

Where $z_i$ are the data, $\alpha$ is a parameter to control the smoothness, and $K_\alpha$ is the kernel density function passed as `kernel`.

<!--
BEGIN QUESTION
name: q2c1
manual: false
points: 2
-->

In [21]:
def kde(kernel, alpha, x, data):
    """
    Compute the kernel density estimate for the single query point x.

    Args:
        kernel: a kernel function with 3 parameters: alpha, x, data
        alpha: the smoothing parameter to pass to the kernel
        x: a single query point (in one dimension)
        data: a numpy array of data points

    Returns:
        The smoothed estimate at the query point x
    """
    ...
    
    ...

In [ ]:
grader.check("q2c1")

Assuming you implemented `kde` correctly, the code below should generate the `kde` of the log of the income data as before.

In [23]:
df['trans_inc'] = np.log10(df['inc'])
xs = np.linspace(df['trans_inc'].min(), df['trans_inc'].max(), 1000)
curve = [kde(gaussian_kernel, alpha, x, df['trans_inc']) for x in xs]
plt.hist(df['trans_inc'], density=True, color='orange')
plt.title('World Bank Gross National Income Per Capita')
plt.xlabel('Log Gross national income per capita, Atlas method: $: 2016');
plt.plot(xs, curve, 'k-');

And the code below should show a 3 x 3 set of plots showing the output of the kde for different `alpha` values.

In [24]:
plt.figure(figsize=(15,15))
alphas = np.arange(0.2, 2.0, 0.2)
for i, alpha in enumerate(alphas):
    plt.subplot(3, 3, i+1)
    xs = np.linspace(df['trans_inc'].min(), df['trans_inc'].max(), 1000)
    curve = [kde(gaussian_kernel, alpha, x, df['trans_inc']) for x in xs]
    plt.hist(df['trans_inc'], density=True, color='orange')
    plt.plot(xs, curve, 'k-')
plt.show()

Let's take a look at another kernel, the Boxcar kernel.

In [25]:
def boxcar_kernel(alpha, x, z):
    return (((x-z)>=-alpha/2)&((x-z)<=alpha/2))/alpha

Run the cell below to enable interactive plots. It should give you a green 'OK' when it's finished.

In [26]:
from ipywidgets import interact
!jupyter nbextension enable --py widgetsnbextension

Now, we can plot the Boxcar and Gaussian kernel functions to see what they look like.

In [27]:
x = np.linspace(-10,10,1000)
def f(alpha):
    plt.plot(x, boxcar_kernel(alpha,x,0), label='Boxcar')
    plt.plot(x, gaussian_kernel(alpha,x,0), label='Gaussian')
    plt.legend(title='Kernel Function')
    plt.show()
interact(f, alpha=(1,10,0.1));

Using the interactive plot below compare the the two kernel techniques:  (Generating the KDE plot is slow, so you may expect some latency after you move the slider)

In [28]:
xs = np.linspace(df['trans_inc'].min(), df['trans_inc'].max(), 1000)
def f(alpha_g, alpha_b):
    plt.hist(df['trans_inc'], density=True, color='orange')
    g_curve = [kde(gaussian_kernel, alpha_g, x, df['trans_inc']) for x in xs]
    plt.plot(xs, g_curve, 'k-', label='Gaussian')
    b_curve = [kde(boxcar_kernel, alpha_b, x, df['trans_inc']) for x in xs]
    plt.plot(xs, b_curve, 'r-', label='Boxcar')
    plt.legend(title='Kernel Function')
    plt.show()
interact(f, alpha_g=(0.01,.5,0.01), alpha_b=(0.01,3,0.1));

<!-- BEGIN QUESTION -->

Briefly compare and contrast the Gaussian and Boxcar kernels in the cell below.

<!--
BEGIN QUESTION
name: q2c2
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



_Intentionally Blank_

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()